In [1]:
using DiffEqSensitivity, OrdinaryDiffEq, DiffEqParamEstim, ParameterizedFunctions, BlackBoxOptim, NLopt, Calculus, ForwardDiff, Optim

In [2]:
include("/Users/vaibhav/DiffEqParamEstim.jl/src/localsaparamestim.jl")

costfunc_gradient (generic function with 1 method)

In [3]:
fitz = @ode_def_nohes FitzhughNagumo begin
  dv = v - v^3/3 -w + 0.5
  dw = 0.08*(v +  a - 0.8*w)
end a

(::FitzhughNagumo{getfield(Main, Symbol("##5#11")),getfield(Main, Symbol("##6#12")),getfield(Main, Symbol("##7#13")),getfield(Main, Symbol("##8#14")),getfield(Main, Symbol("##9#15")),getfield(Main, Symbol("##10#16")),Expr,Expr}) (generic function with 2 methods)

In [4]:
p = [0.7]              # Parameters used to construct the dataset
r0 = [1.0; 1.0]                     # initial value
tspan = (0.0, 3.0)                 # sample of 3000 observations over the (0,30) timespan
prob = ODEProblem(fitz, r0, tspan,p)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: [1.0, 1.0]

In [5]:
t = collect(range(0, stop=3.0, length=1000))
data = Array(solve(prob,Tsit5(),saveat=t,abstol=1e-3,reltol=1e-3))

2×1000 Array{Float64,2}:
 1.0  1.0005   1.001    1.0015   …  1.11337  1.11316  1.11296  1.11275
 1.0  1.00022  1.00043  1.00065     1.222    1.2222   1.2224   1.2226 

In [6]:
ourprob = ODELocalSensitivityProblem(fitz,r0,tspan,p)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: [1.0, 1.0, 0.0, 0.0]

In [7]:
oursol = solve(ourprob,Tsit5(),saveat=t,abstol=1e-6,reltol=1e-6)

retcode: Success
Interpolation: 1st order linear
t: 1000-element Array{Float64,1}:
 0.0                 
 0.003003003003003003
 0.006006006006006006
 0.009009009009009009
 0.012012012012012012
 0.015015015015015015
 0.018018018018018018
 0.021021021021021023
 0.024024024024024024
 0.02702702702702703 
 0.03003003003003003 
 0.03303303303303303 
 0.036036036036036036
 ⋮                   
 2.966966966966967   
 2.96996996996997    
 2.972972972972973   
 2.975975975975976   
 2.978978978978979   
 2.981981981981982   
 2.984984984984985   
 2.987987987987988   
 2.990990990990991   
 2.993993993993994   
 2.996996996996997   
 3.0                 
u: 1000-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 0.0, 0.0]                       
 [1.0005, 1.00022, -3.60698e-7, 0.000240217]
 [1.001, 1.00043, -1.4427e-6, 0.000480388]  
 [1.0015, 1.00065, -3.24584e-6, 0.000720512]
 [1.002, 1.00087, -5.76998e-6, 0.00096059]  
 [1.00249, 1.00108, -9.01496e-6, 0.00120062]
 [1.00299, 1.0013, -1.29806e-5, 

In [8]:
myl2loss(oursol,data)

1.4591534767036364e-7

In [9]:
size(oursol)

(4, 1000)

In [13]:
out = zeros(eltype(oursol),1)

1-element Array{Float64,1}:
 0.0

In [17]:
myl2lossgradient!(out,oursol,data,1)

In [18]:
out

1-element Array{Float64,1}:
 0.00010425580660985847

In [23]:
p = [0.8]
costfunc_gradient(p,out)

0.2846344980787316

In [31]:
p = [0.5]
@benchmark begin
    out = Array{Float64}(undef,2)
    costfunc_gradient(p,out)
end

BenchmarkTools.Trial: 
  memory estimate:  575.72 KiB
  allocs estimate:  2886
  --------------
  minimum time:     498.923 μs (0.00% GC)
  median time:      544.177 μs (0.00% GC)
  mean time:        632.375 μs (10.72% GC)
  maximum time:     5.007 ms (81.26% GC)
  --------------
  samples:          7851
  evals/sample:     1

In [32]:
@benchmark Calculus.finite_difference!(costfunc,p,out,:central)

BenchmarkTools.Trial: 
  memory estimate:  544.19 KiB
  allocs estimate:  2887
  --------------
  minimum time:     457.660 μs (0.00% GC)
  median time:      495.616 μs (0.00% GC)
  mean time:        563.601 μs (10.87% GC)
  maximum time:     3.427 ms (84.31% GC)
  --------------
  samples:          8843
  evals/sample:     1

In [24]:
opt = Opt(:LD_MMA, 1)
min_objective!(opt, costfunc_gradient)
lower_bounds!(opt,[0.0])
upper_bounds!(opt,[1.0])
xtol_rel!(opt,1e-12)
maxeval!(opt, 10000)
(minf,minx,ret) = NLopt.optimize(opt,[0.5])

(1.4581897986914204e-7, [0.699998], :XTOL_REACHED)

In [29]:
function l!(grad,p)
    costfunc_gradient(p,grad)
end

l! (generic function with 1 method)

In [30]:
Optim.optimize(costfunc, l!, [0.4], ConjugateGradient())

Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [0.4]
 * Minimizer: [0.6999981513469326]
 * Minimum: 1.458190e-07
 * Iterations: 5
 * Convergence: true
   * |x - x'| ≤ 0.0e+00: false 
     |x - x'| = 2.64e-10 
   * |f(x) - f(x')| ≤ 0.0e+00 |f(x)|: false
     |f(x) - f(x')| = 5.73e-10 |f(x)|
   * |g(x)| ≤ 1.0e-08: true 
     |g(x)| = 2.21e-13 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 13
 * Gradient Calls: 10